In [12]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path

from natsort import natsorted
from dotenv import find_dotenv


root_dir = os.path.dirname(find_dotenv())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
root_dir

'/home/bombra/library/padme'

In [14]:
from src import DMD
from src import snapshots_assembly
from src import (
    load_notebook_params,
    generate_paths,
    download_dataset,
    unpack_kaggle_dataset,
)

In [15]:
notebook_params = load_notebook_params()

Before downloading dataset, you need to create an account on Kaggle. Instructions for using the Kaggle API can be seen at https://www.kaggle.com/docs/api
This dataset contains 991MB and two versions of the simulation covid spread lombardy etc

In [16]:
example_dataset = "seird"

dict_paths = generate_paths(example_dataset, root_dir, notebook_params)
dict_paths

download_dataset(dict_paths, example_dataset, notebook_params, FORCE_DOWNLOAD=False)
unpack_kaggle_dataset(dict_paths)

{'complete_filepath': PosixPath('/home/bombra/library/padme/data/00_raw'),
 'complete_filename': PosixPath('/home/bombra/library/padme/data/00_raw/covid-19-spread-in-lombardy-italy.zip'),
 'snapshots_filepath': PosixPath('/home/bombra/library/padme/data/00_raw/covid-seird-lombardy-libmesh/covid-seird-lombardy-libmesh')}

We need to unzip etc etc etc
points to data folder

Let's instantiate the dataset etc etc etc

In [21]:
os_walk_files = next(os.walk(dict_paths["snapshots_filepath"]))
folders = natsorted(os_walk_files[1])

filenames = [
    dict_paths["snapshots_filepath"]
    / Path(snapshot_folder)
    / Path(f"out_1_000_{str(i).zfill(5)}.h5")
    for i, snapshot_folder in enumerate(folders)
]

In [22]:
filenames[:5]

[PosixPath('/home/bombra/library/padme/data/00_raw/covid-seird-lombardy-libmesh/covid-seird-lombardy-libmesh/step0/out_1_000_00000.h5'),
 PosixPath('/home/bombra/library/padme/data/00_raw/covid-seird-lombardy-libmesh/covid-seird-lombardy-libmesh/step1/out_1_000_00001.h5'),
 PosixPath('/home/bombra/library/padme/data/00_raw/covid-seird-lombardy-libmesh/covid-seird-lombardy-libmesh/step2/out_1_000_00002.h5'),
 PosixPath('/home/bombra/library/padme/data/00_raw/covid-seird-lombardy-libmesh/covid-seird-lombardy-libmesh/step3/out_1_000_00003.h5'),
 PosixPath('/home/bombra/library/padme/data/00_raw/covid-seird-lombardy-libmesh/covid-seird-lombardy-libmesh/step4/out_1_000_00004.h5')]

In [23]:
snapshot_ingestion_parameters = {
    "filenames": filenames,
    "dataset": "s",
}

In [24]:
dataset = snapshots_assembly(
    "h5_libmesh", snapshot_ingestion_parameters=snapshot_ingestion_parameters
)

2023-04-16 16:56:43,059 - src.preprocessing.build_snapshots - INFO - Starting choice of file type:
2023-04-16 16:56:43,060 - src.preprocessing.build_snapshots - INFO - libMesh/EdgeCFD HDF5 file selected.


In [25]:
dataset.shape

(13158, 482)

In [26]:
example_dataset = "sird"

dict_paths = generate_paths(example_dataset, root_dir, notebook_params)
dict_paths

download_dataset(dict_paths, example_dataset, notebook_params, FORCE_DOWNLOAD=False)
unpack_kaggle_dataset(dict_paths)

In [30]:
os_walk_files = next(os.walk(dict_paths["snapshots_filepath"]))
files = natsorted(os_walk_files[2])

filenames = [
    dict_paths["snapshots_filepath"] / Path(f"{str(i)}infected.vtk")
    for i, _ in zip(range(4, 480, 4), files)
]

In [31]:
filenames[:5]

[PosixPath('/home/bombra/library/padme/data/00_raw/covid-sird-lombardy-freefem/covid-sird-lombardy-freefem/4infected.vtk'),
 PosixPath('/home/bombra/library/padme/data/00_raw/covid-sird-lombardy-freefem/covid-sird-lombardy-freefem/8infected.vtk'),
 PosixPath('/home/bombra/library/padme/data/00_raw/covid-sird-lombardy-freefem/covid-sird-lombardy-freefem/12infected.vtk'),
 PosixPath('/home/bombra/library/padme/data/00_raw/covid-sird-lombardy-freefem/covid-sird-lombardy-freefem/16infected.vtk'),
 PosixPath('/home/bombra/library/padme/data/00_raw/covid-sird-lombardy-freefem/covid-sird-lombardy-freefem/20infected.vtk')]

In [32]:
filenames[-5:]

[PosixPath('/home/bombra/library/padme/data/00_raw/covid-sird-lombardy-freefem/covid-sird-lombardy-freefem/460infected.vtk'),
 PosixPath('/home/bombra/library/padme/data/00_raw/covid-sird-lombardy-freefem/covid-sird-lombardy-freefem/464infected.vtk'),
 PosixPath('/home/bombra/library/padme/data/00_raw/covid-sird-lombardy-freefem/covid-sird-lombardy-freefem/468infected.vtk'),
 PosixPath('/home/bombra/library/padme/data/00_raw/covid-sird-lombardy-freefem/covid-sird-lombardy-freefem/472infected.vtk'),
 PosixPath('/home/bombra/library/padme/data/00_raw/covid-sird-lombardy-freefem/covid-sird-lombardy-freefem/476infected.vtk')]

In [33]:
snapshot_ingestion_parameters = {
    "filenames": filenames,
    "starting_line": 125939,
    "ending_line": 210239,
}

In [34]:
dataset = snapshots_assembly(
    "vtk_freefem", snapshot_ingestion_parameters=snapshot_ingestion_parameters
)

2023-04-16 16:59:50,442 - src.preprocessing.build_snapshots - INFO - Starting choice of file type:
2023-04-16 16:59:50,443 - src.preprocessing.build_snapshots - INFO - FreeFem++ vtk file selected.


In [35]:
dmd_parameters = {
    "factorization_algorithm": "randomized_svd",
    "basis_vectors": 50,
    "randomized_svd_parameters": {
        "power_iterations": 1,
        "oversampling": 20,
    },
    "starting_step": 20,
    "dt_simulation": 0.05,
}

In [36]:
dmd = DMD(dataset, dmd_parameters)

In [37]:
dmd.factorization()

In [38]:
dmd.dmd_approximation["s"]

array([1.07649634e+04, 1.38118735e+03, 1.72657326e+02, 6.35171941e+01,
       1.98410898e+01, 1.01267016e+01, 4.87663858e+00, 3.07436629e+00,
       1.16294959e+00, 6.69079356e-01, 3.68375673e-01, 2.94146842e-01,
       1.71937259e-01, 8.56296157e-02, 6.12975687e-02, 7.02276972e-03,
       2.46134242e-03, 7.26166294e-04, 5.37647098e-04, 4.63511683e-04,
       3.88596940e-04, 3.03577439e-04, 2.82226113e-04, 2.62287883e-04,
       2.41581669e-04, 2.26978726e-04, 2.08615192e-04, 2.03556807e-04,
       1.93375883e-04, 1.86029158e-04, 1.76899071e-04, 1.70248770e-04,
       1.46752813e-04, 1.44910333e-04, 1.32680714e-04, 1.24961398e-04,
       1.17451283e-04, 1.10451526e-04, 1.06375751e-04, 9.14862257e-05,
       7.39796044e-05, 7.09598365e-05, 6.17926082e-05, 4.75911113e-05,
       3.76060746e-05, 3.30560030e-05, 2.23307815e-05, 2.15716855e-05,
       2.06865128e-05, 1.77037217e-05])

In [39]:
dmd.dmd_core()

In [42]:
import numpy as np


def compute_frobenius_norm(mat_a, mat_b):
    return np.linalg.norm(mat_a - mat_b) / np.linalg.norm(mat_a)

In [43]:
compute_frobenius_norm(dmd.snapshots_matrix, dmd.dmd_approximation["dmd_matrix"])

0.7325661673259111

In [44]:
dmd.dmd_approximation.keys()

dict_keys(['u', 's', 'vt', 'eigenvals_original', 'eigenvals_processed', 'dmd_matrix'])